# **Hotel Recommendation System**



In [ ]:
!cd /content/drive/MyDrive/Colab_me/DS300

In [ ]:
!pwd

/content/drive/MyDrive/DS300_DoAn/data


In [ ]:
# 1. Install gcc if it is not installed already. On Ubuntu, this could done by using the command
# sudo apt install gcc

# 2. Create and activate a new conda environment
!conda create -n <environment_name> python=3.9
!conda activate <environment_name>

# 3. Install the core recommenders package. It can run all the CPU notebooks.
!pip install recommenders

# 4. create a Jupyter kernel
!python -m ipykernel install --user --name <environment_name> --display-name <kernel_name>

# 5. Clone this repo within VSCode or using command line:
# !git clone https://github.com/recommenders-team/recommenders.git

# 6. Within VSCode:
#   a. Open a notebook, e.g., examples/00_quick_start/sar_movielens.ipynb;
#   b. Select Jupyter kernel <kernel_name>;
#   c. Run the notebook.

/bin/bash: line 1: environment_name: No such file or directory
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `conda activate <environment_name>'
ERROR: Ignored the following versions that require a different python version: 0.6.0 Requires-Python >=3.6, <=3.8; 0.7.0 Requires-Python >=3.6, <3.8; 1.0.0 Requires-Python >=3.6, <3.9; 1.1.0 Requires-Python >=3.6, <3.10; 1.1.1 Requires-Python >=3.6, <3.10
ERROR: Could not find a version that satisfies the requirement recommenders (from versions: none)
ERROR: No matching distribution found for recommenders
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `python -m ipykernel install --user --name <environment_name> --display-name <kernel_name>'


## **Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time
import datetime
import random
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from tqdm import tqdm
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from numpy import dot
from numpy import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

sns.set()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **Read dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/DS300_DoAn/data

/content/drive/MyDrive/DS300_DoAn/data


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_history.csv')
convert_timestamp = lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple())

df['timestamp'] = df['Date'].apply(convert_timestamp)

In [ ]:
df

,IDuser,IDhotel,Rating,Date,timestamp
0,5140,277,6.3,2011-10-15,1.318637e+09
1,2059,256,6.7,2011-11-16,1.321402e+09
2,7845,171,7.3,2012-01-05,1.325722e+09
3,9689,182,7.0,2012-01-05,1.325722e+09
4,3040,150,6.0,2012-01-06,1.325808e+09
...,...,...,...,...,...
18264,5961,183,8.0,2023-12-08,1.701994e+09
18265,1331,5,7.0,2023-12-08,1.701994e+09
18266,5405,106,10.0,2023-12-09,1.702080e+09
18267,5015,70,7.0,2023-12-09,1.702080e+09


In [ ]:
df = df[df.IDuser.map(df.IDuser.value_counts()) > 4]
df

,IDuser,IDhotel,Rating,Date,timestamp
8,3827,378,6.0,2012-01-08,1.325981e+09
10,5272,182,6.0,2012-02-07,1.328573e+09
11,5822,308,6.0,2012-02-08,1.328659e+09
14,5754,141,6.7,2012-03-08,1.331165e+09
15,5961,334,6.7,2012-03-16,1.331856e+09
...,...,...,...,...,...
18247,5668,103,8.0,2023-12-07,1.701907e+09
18255,8139,47,10.0,2023-12-07,1.701907e+09
18256,4080,140,8.0,2023-12-08,1.701994e+09
18263,5857,571,4.0,2023-12-08,1.701994e+09


In [ ]:
ratings = df.apply(lambda row: (row.IDuser, row.IDhotel, row.Rating), axis=1).tolist()

## **Collaborative Filtering**

In [ ]:
!pip install cornac
!pip3 install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 14.2 MB/s eta 0:00:00


In [ ]:
# Code nay de lay do do
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import PMF, BiVAECF, MF, BPR, BiVAECF, EASE, IBPR, SKMeans
from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP

In [ ]:
rs = RatioSplit(data=ratings, test_size=0.2, rating_threshold=8.5, seed=42)

/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 162 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.10/dist-packages/cornac/data/dataset.py:339: UserWarning: 8 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [ ]:
# define metrics to evaluate the models
metrics = [Precision(k=5), Recall(k=5), NDCG(k=5), Precision(k=10), Recall(k=10), NDCG(k=10), AUC(), MAP()]

In [ ]:
# initialize models, here we are comparing: Biased MF, PMF, and BPR
pmf = PMF(k=40, max_iter=100, learning_rate=0.001, lambda_reg=0.001, seed=42)
bpr = BPR(seed=42)
bibaecf = BiVAECF(k=50, encoder_structure=[100], act_fn="tanh", likelihood="pois", n_epochs=100, batch_size=32,
                  learning_rate=0.001,
                  seed=42,
                  verbose=True)
models = [pmf, bpr, bibaecf]

# put it together in an experiment!
cornac.Experiment(eval_method=rs, models=models, metrics=metrics, user_based=True).run()

  0%|          | 0/100 [00:00<?, ?it/s]


TEST:
...
        |    AUC |    MAP | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------- + ------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
PMF     | 0.6423 | 0.0460 |  0.0470 | 0.0402 |       0.0126 |      0.0174 |    0.0671 |   0.0478 |    0.5771 |   0.7519
BPR     | 0.7620 | 0.0656 |  0.0801 | 0.0548 |       0.0243 |      0.0261 |    0.1550 |   0.0847 |    0.1530 |   0.2779
BiVAECF | 0.7569 | 0.0651 |  0.0740 | 0.0518 |       0.0222 |      0.0252 |    0.1306 |   0.0660 |   48.2763 |   0.3019



In [ ]:
ease_original = cornac.models.EASE(
    lamb=500,
    name="EASEᴿ (B>0)",
    posB=True
)

ease_all = cornac.models.EASE(
    lamb=500,
    name="EASEᴿ (B>-∞)",
    posB=False
)

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=rs,
    models=[ease_original, ease_all],
    metrics=metrics,
    user_based=True, #If `False`, results will be averaged over the number of ratings.
    save_dir=None
).run()


TEST:
...
             |    AUC |    MAP | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------------ + ------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
EASEᴿ (B>0)  | 0.7016 | 0.0653 |  0.0724 | 0.0541 |       0.0226 |      0.0252 |    0.1159 |   0.0642 |    0.0442 |   0.3351
EASEᴿ (B>-∞) | 0.5938 | 0.0652 |  0.0740 | 0.0614 |       0.0209 |      0.0287 |    0.1089 |   0.0667 |    0.0278 |   0.3280



In [ ]:
# Instantiate the NGCF model
ngcf = cornac.models.NGCF(
    seed=123,
    num_epochs=100,
    emb_size=64,
    layer_sizes=[64, 64, 64],
    dropout_rates=[0.1, 0.1, 0.1],
    early_stopping={"min_delta": 1e-4, "patience": 50},
    batch_size=1024,
    learning_rate=0.001,
    lambda_reg=1e-5,
    verbose=True,
)

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=rs,
    models=[ngcf],
    metrics=metrics,
    user_based=True,
).run()

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


Training:   0%|          | 0/100 [00:00<?, ?iter/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]


TEST:
...
     |    AUC |    MAP | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
---- + ------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
NGCF | 0.6942 | 0.0502 |  0.0465 | 0.0377 |       0.0122 |      0.0157 |    0.0738 |   0.0492 |   87.6239 |   0.4015



In [ ]:
# Instantiate the VAECF model
vaecf = cornac.models.VAECF(
    k=10,
    autoencoder_structure=[20],
    act_fn="tanh",
    likelihood="mult",
    n_epochs=100,
    batch_size=100,
    learning_rate=0.001,
    beta=1.0,
    seed=123,
    use_gpu=True,
    verbose=True,
)

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=rs,
    models=[vaecf],
    metrics=metrics,
    user_based=True,
).run()

  0%|          | 0/100 [00:00<?, ?it/s]


TEST:
...
      |    AUC |    MAP | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
----- + ------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
VAECF | 0.7615 | 0.0615 |  0.0733 | 0.0492 |       0.0230 |      0.0270 |    0.1430 |   0.0745 |    3.4934 |   0.4424



In [ ]:
# Instantiate a IBPR recommender model.
ibpr = IBPR(k=40, verbose=True)

# Instantiate and then run an experiment.
cornac.Experiment(
    eval_method=rs, models=[ibpr], metrics=metrics, user_based=True
).run()

Epoch 1/100:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 2/100:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 3/100:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 4/100:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 5/100:   0%|          | 0/37 [00:00<?, ?it/s]

...

Epoch 97/100:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 98/100:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 99/100:   0%|          | 0/37 [00:00<?, ?it/s]

Epoch 100/100:   0%|          | 0/37 [00:00<?, ?it/s]


TEST:
...
     |    AUC |    MAP | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
---- + ------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
IBPR | 0.7362 | 0.0588 |  0.0606 | 0.0489 |       0.0174 |      0.0235 |    0.1021 |   0.0702 |   43.0695 |   0.9148



In [ ]:
backend = "tensorflow"  # or 'pytorch'

# Instantiate the recommender models to be compared
gmf = cornac.models.GMF(
    num_factors=8,
    num_epochs=50,
    learner="adam",
    backend=backend,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
mlp = cornac.models.MLP(
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    backend=backend,
    num_epochs=50,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
neumf = cornac.models.NeuMF(
    num_factors=8,
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    backend=backend,
    num_epochs=50,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=rs,
    models=[
        gmf,
        mlp,
        neumf],
    metrics=metrics,
).run()

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]


TEST:
...
      |    AUC |    MAP | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
----- + ------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
GMF   | 0.7614 | 0.0643 |  0.0791 | 0.0539 |       0.0243 |      0.0261 |    0.1550 |   0.0847 |  188.6608 |   0.5861
MLP   | 0.7618 | 0.0754 |  0.0861 | 0.0642 |       0.0235 |      0.0270 |    0.1463 |   0.0874 |  193.6682 |   0.8067
NeuMF | 0.7627 | 0.0667 |  0.0793 | 0.0551 |       0.0230 |      0.0261 |    0.1586 |   0.0893 |  195.2046 |   0.6072



In [ ]:
skm = cornac.models.SKMeans(
   k=5,
   max_iter=50,
   tol=1e-10,
   seed=123
)

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=rs,
    models=[skm],
    metrics=metrics,
    user_based=False,
).run()

Iter 1, likelihood: 98.087356
Iter 2, likelihood: 108.556141
Iter 3, likelihood: 113.251747
Iter 4, likelihood: 115.974237
Iter 5, likelihood: 117.286404
Iter 6, likelihood: 118.027412
Iter 7, likelihood: 118.507122
Iter 8, likelihood: 118.886714
Iter 9, likelihood: 119.277516
Iter 10, likelihood: 119.444897
Iter 11, likelihood: 119.615961
Iter 12, likelihood: 119.997324
Iter 13, likelihood: 120.407041
Iter 14, likelihood: 120.625238
Iter 15, likelihood: 120.707895
Iter 16, likelihood: 120.828165
Iter 17, likelihood: 120.921069
Iter 18, likelihood: 121.092174
Iter 19, likelihood: 121.242338
Iter 20, likelihood: 121.328821

TEST:
...
        |    AUC |    MAP | NDCG@10 | NDCG@5 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------- + ------ + ------ + ------- + ------ + ------------ + ----------- + --------- + -------- + --------- + --------
Skmeans | 0.7545 | 0.0716 |  0.0811 | 0.0579 |       0.0235 |      0.0261 |    0.1448 |   0.0803 |    0.2169 |   0.5444